# Clusteranalyse

Schritte zur Vorbereitung, später evtl. schon im Preprossessing erledigt:

Format
- Datumsspalte in Jahr, Monat, Tag trennen

Daten entfernen
- **Zeilen** aus 2014 und 2025 entfernen, weil zu wenige Datenpunkte
- **Spalten** mit zu vielen fehlenden Werten entfernen (cut-off: 53% missing values)
- Alle Einträge für **Tehran** entfernen, weil die Stadt durch ihre extrem hohen Schadstoffwerte die Clusteranalyse verzerrt

Aktuell auch noch relevant, aber im großen Datensatz vielleicht nicht mehr
- Bestimmen, in welchen der 95 Städte für **alle Schadstoffe** Messwerte vorliegen --> 54 Städte

Clusteranalyse
- StandardScaler und KMeans importieren
- Liste der Schadstoffe definieren (pollutants)
- gruppierten ("City") und reduzierten ("dropna") mit Mittelwerten ("mean") df nach Städten und Schadstoffen für als Datengrundlage der Clusteranalyse erstellen (**df_cluster**). df-cluster hat 53 Zeilen (Städte) und 6 Spalten (Schadstoffe)
- Daten skalieren (**df_cluster_scaled**)
- mit Ellbow-Methode optimale Clusteranzahl bestimmen --> 5 Cluster (Silhouette wurde auch getestet, brachte keinen Mehrwert)
- mit Kmeans Clusterzuordnung durchführen (**df_cluster_numbers**)

In [ ]:
# imports
import pandas as pd
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
%matplotlib inline

In [ ]:
# Settings for displaying floats
pd.set_option('display.float_format', '{:,.2f}'.format)

In [ ]:
df = pd.read_csv("/Users/mareikekeller/air_quality/data/cleaned_data.csv")
df.head()

In [ ]:
# Data preparation: Manipulating the 'Date' column

# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Split 'Date' column into 'year', 'month' and 'day'
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day

# Remove 'Date' column
if 'Date' in df.columns:
    df.drop(columns=['Date'], inplace=True)

In [ ]:
# Daten für 2014 & 2025 entfernen, weil zu wenige Datenpunkte

df = df[(df["year"] > 2014) & (df["year"] < 2025)]

In [ ]:
# Tehran komplett entfernen, weil die Schadstoffwerte zu sehr von allen übrigen Städten abweichen
df = df[df["City"] != "Tehran"]

In [ ]:
print("Tehran noch im DataFrame?", "Tehran" in df["City"].values)

In [ ]:
df['City'].nunique()

In [ ]:
# Display the first 5 rows of the dataframe
df.head()

In [ ]:
df.shape

In [ ]:
# Spalten mit zu vielen fehlenden Werten entfernen

# Berechnen, wie viele Prozent der Werte pro Spalte fehlen
missing_percent = df.isna().mean() * 100  

# Spalten auswählen, die weniger als 50% fehlende Werte haben
df_cleaned = df.loc[:, missing_percent <= 53]

# Ergebnis ausgeben
print(f"Anzahl der entfernten Spalten: {df.shape[1] - df_cleaned.shape[1]}")
print("Übrige Spalten:", df_cleaned.columns)


In [ ]:
df_cleaned["City"].nunique()

In [ ]:
# # Mit Heatmap herausfinden, welche Städte für welche Schadstoffe fehlende Werte haben

# # Liste der Schadstoff-Features für das Clustering
# pollutants = ["co", "no2", "o3", "so2", "pm10", "pm25"]

# # DataFrame mit den Schadstoffen pro Stadt erstellen
# df_missing = df_cleaned.groupby("City")[pollutants].mean()

# # Boolean-Maske für fehlende Werte erstellen (True = fehlend, False = vorhanden)
# missing_data = df_missing.isna()

# # Größe der Grafik anpassen
# plt.figure(figsize=(8, 20))

# # Heatmap zeichnen (dunklere Farben = mehr fehlende Werte)
# sns.heatmap(missing_data, cmap="coolwarm", cbar=False, linewidths=0.5)

# # Achsentitel setzen
# plt.xlabel("Schadstoffe")
# plt.ylabel("Städte")
# plt.title("Heatmap der fehlenden Werte pro Stadt und Schadstoff")


In [ ]:
# missing_per_city = df_missing.isna().sum(axis=1)
# missing_per_city_sorted = missing_per_city.sort_values(ascending=False)
# print(missing_per_city_sorted.to_string())


## Code für Clusteranalyse (K-Means) zur Schadstoffbelastung

In [ ]:
# Clusteranalyse zur Schadstoffbelastung

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Liste der Schadstoff-Features für das Clustering
pollutants = ["co", "no2", "o3", "so2", "pm10", "pm25"]

# Durchschnittliche Schadstoffwerte pro Stadt berechnen
df_cluster = df_cleaned.groupby("City")[pollutants].mean().dropna()



In [ ]:
df_cluster.head()

In [ ]:
df_cluster.shape

In [ ]:
# Daten skalieren (K-Means ist empfindlich gegenüber unterschiedlichen Skalen)
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_cluster)

# Ergebnis als DataFrame zurückgeben
df_cluster_scaled = pd.DataFrame(df_scaled, index=df_cluster.index, columns=pollutants)

# Überprüfen, ob die Daten korrekt vorbereitet sind
df_cluster_scaled.head()

In [ ]:
# Teste verschiedene Clusterzahlen (k = 1 bis 10)
inertia = []
k_values = range(1, 31)

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(df_cluster_scaled)
    inertia.append(kmeans.inertia_)  # Speichert den Fehler (Inertia)

# Elbow-Plot erstellen
plt.figure(figsize=(8, 5))
plt.plot(k_values, inertia, marker='o', linestyle='-')
plt.xlabel("Anzahl der Cluster (k)")
plt.ylabel("Inertia (Fehler)")
plt.title("Elbow-Methode zur Bestimmung der optimalen Clusterzahl")
plt.grid(True);

In [ ]:
# K-Means-Clustering
kmeans = KMeans(n_clusters=6, random_state=42, n_init=10)
df_cluster_scaled["Cluster"] = kmeans.fit_predict(df_cluster_scaled)

# Neue Cluster-Zuordnung der Städte anzeigen
df_cluster_numbers = df_cluster_scaled[["Cluster"]].sort_values(by="Cluster")
df_cluster_numbers

In [ ]:
df_cluster_scaled

In [ ]:
df_cluster = df_cluster_scaled.copy()

In [ ]:
df_cluster.to_csv("df_cluster.csv", index=True)  # Index speichern, damit die Städtenamen erhalten bleiben


In [ ]:
# # Delhi wird bei 5 und 6 Clustern von KMeans als eigenes Cluster isoliert.
# # Test, ob DBSCAN eine andere Clusterbildung erzielt

# from sklearn.cluster import DBSCAN

# dbscan = DBSCAN(eps=3.5, min_samples=4)  # Parameter ggf. anpassen
# df_cluster["Cluster_DBSCAN"] = dbscan.fit_predict(df_scaled)

In [ ]:
# df_cluster["Cluster_DBSCAN"].value_counts()


In [ ]:
# df_cluster[df_cluster.index == "Delhi"]

Entscheidung: Tehran war schon rausgenommen, weil es ein eigenes Cluster bildet. Ohne Tehran bildet Delhi ein eigenes Cluster. Das lassen wir jetzt erst mal so.

## Zusammenführung Geodaten und Clusternummern

In [ ]:
# import os
# print(os.getcwd())

In [ ]:
df_geodata = pd.read_csv("df_geodata.csv")
df_geodata.head() 

In [ ]:
df_cluster_numbers.head(20)

In [ ]:
print(df_cluster_numbers)

In [ ]:
print(df_cluster_numbers.index[:10])  # Zeigt die ersten 10 Indexwerte von df_cluster_numbers
print(df_geodata["City"].unique()[:10])

In [ ]:
df_geodata["City"] = df_geodata["City"].str.strip().str.lower()
df_cluster_numbers.index = df_cluster_numbers.index.str.strip().str.lower()

In [ ]:
df_geodata = df_geodata.merge(df_cluster_numbers[["Cluster"]], left_on="City", right_index=True, how="left")


In [ ]:
df_geodata.head()

In [ ]:
df_geodata = df_geodata.dropna(subset=["Cluster"])

In [ ]:
df_geodata

## Cluster farbig plotten

In [ ]:


import geodatasets
import matplotlib.pyplot as plt

# Weltkarte laden
world = gpd.read_file(geodatasets.get_path("naturalearth.land"))

# Städte aus df_geodata in einen GeoDataFrame umwandeln
gdf_cities = gpd.GeoDataFrame(df_geodata, 
                              geometry=gpd.points_from_xy(df_geodata["Longitude"], df_geodata["Latitude"]))


In [ ]:
import matplotlib.colors as mcolors

# Eigene 5-Farben-Palette aus 'tab10' extrahieren
custom_cmap = mcolors.ListedColormap(plt.get_cmap("tab10").colors[:5])

In [ ]:
custom_cmap

In [ ]:
# Plot erstellen
fig, ax = plt.subplots(figsize=(15, 12))

# Umrisse der Weltkarte plotten
world.plot(ax=ax, color='white', edgecolor='black')

# Städte als farbige Punkte nach Cluster einfärben
scatter = gdf_cities.plot(column="Cluster", cmap=custom_cmap, ax=ax, markersize=30, legend=True, alpha=1.0)

# Titel & Achsen
plt.title("Clusteranalyse der Städte basierend auf Luftverschmutzung")
plt.xlabel("Längengrad")
plt.ylabel("Breitengrad")

plt.savefig("../Images/cluster_plot.png", dpi=300, bbox_inches="tight")

plt.show()



In [ ]:
# plt.savefig("cluster_plot.png", dpi=300, bbox_inches="tight")
plt.savefig("../Images/cluster_plot.png", dpi=300, bbox_inches="tight")
